In [1]:
import azureml
import azureml.core
from azureml.core import Workspace, Run, Experiment
from azureml.core.model import Model

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.6


### Getting the workspace
Can be done from a local json config file or using one straight from the Azure Service:

In [2]:
# ws = Workspace.from_config()

# OR

ws = Workspace.get(name='mlworkspace',
                      subscription_id='08cb84a2-122a-4813-a67e-88d3c2530ea1',    
                      resource_group='ml-openhack-rg')

# ws.get_details()
# OR
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\n')

mlworkspace
eastus
ml-openhack-rg
eastus


In [3]:
experimentName = "KerasExperiment"
exp = Experiment(workspace = ws, name = experimentName)

In [5]:
# register model 
modelName = 'kerasModel'
modelPath = "challenge4.h5"
model = Model.register(ws, modelPath, modelName, tags=None, properties=None, description=None)

print("The model has been registered")

Registering model kerasModel
The model has been registered


In [6]:
# Retrieving a model
# exist_ok = True overrides an existing model with the same name
model=Model(ws, modelName)
model.download(target_dir = '.', exist_ok= True)

'challenge4.h5'

In [50]:
%%writefile score.py
import json
import numpy as np
import os
import tensorflow as tf
import keras
from azureml.core.model import Model
from azureml.core import Workspace

def init():
    global model
    modelName = 'kerasModel'
    workspaceName = 'mlworkspace'
    subscriptionId = '08cb84a2-122a-4813-a67e-88d3c2530ea1'
    resourceGroup = 'ml-openhack-rg'
    # retrieve the path to the model file using the model name
    ws = Workspace.get(name = workspaceName,
                       subscription_id = subscriptionId,    
                       resource_group = resourceGroup)
    
    model_path = Model.get_model_path(modelName, version=None, _workspace=ws)
    model = tf.keras.models.load_model(model_path)
    
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    results = model.predict(data)
    return json.dumps(results.tolist())

Overwriting score.py


In [51]:
from azureml.core.conda_dependencies import CondaDependencies 

myEnv = CondaDependencies()
myEnv.add_conda_package("keras")
myEnv.add_conda_package("tensorflow")

envFile = "env.yml"

with open(envFile,"w") as f:
    f.write(myEnv.serialize_to_string())
print("Saved dependency info at", envFile)

Saved dependency info at env.yml


In [52]:
# Just reading the env file:
with open(envFile,"r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
- keras
- tensorflow



In [53]:
# Define config for deploying the container image as an ACI web service
from azureml.core.webservice import AciWebservice

aciConfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "shapes",  "method" : "classification"}, 
                                               description='Service to classify retail items')
print(aciConfig)

In [54]:
# Configure the image
from azureml.core.image import ContainerImage
imageConfig = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="env.yml")
print(imageConfig)

In [55]:
# Deploying to ACI
from azureml.core.webservice import Webservice

# Service Name must be lowercase and with '-' or '.' characters
serviceName = 'keras-service'
modelFileName = 'challenge4.h5'

service = Webservice.deploy(workspace = ws,
                            name = serviceName,
                            deployment_config = aciConfig,
                            model_paths = [modelFileName],
                            image_config = imageConfig)

service.wait_for_deployment(show_output = True)
print(service.state)

Registering model challenge4.h5
Creating image
Image creation operation finished for image keras-service:13, operation "Succeeded"
Creating service
Running..........................................................................................................
TimedOutACI service creation operation finished, operation "TimedOut"
Service creation polling reached terminal state, current service state: Unhealthy
{
  "code": "AciHealthEndpointNotUp",
  "message": "Health endpoint is not up"
}
Unhealthy


In [48]:
from azureml.core.model import Model
import keras
import tensorflow as tf
model_address = Model.get_model_path('kerasModel', version=None, _workspace=ws)
print(model_address)
model = tf.keras.models.load_model(model_address)
print(model)

azureml-models/kerasModel/9/challenge4.h5
